In [26]:
import sys
!{sys.executable} -m pip install -q requests numpy pandas altair ipycytoscape ipywidgets

import numpy as np
from numpy.random import normal
import pandas as pd
import altair as alt
from ipycytoscape import CytoscapeWidget
import ipywidgets as widgets

print("All required packages have been installed and imported successfully.")


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
All required packages have been installed and imported successfully.


In [27]:
cloud_api_base_url = 'https://cloud.getdbt.com/api/v2'
metadata_api_url = 'https://metadata.cloud.getdbt.com/graphql'

api_key = ''
headers = {
    "Accept": "application/json",
    "Authorization": f"Bearer {api_key}"
}

In [28]:
def get_accounts():
    r = requests.get(f'{cloud_api_base_url}/accounts', headers=headers)
    accounts_result = r.json()
    return accounts_result.get('data')

# Fetch accounts
accounts_result = get_accounts()
account_id = accounts_result[0].get("id")

# Display all accounts in a DataFrame
if accounts_result:
    df = pd.DataFrame(accounts_result)
    print("All Accounts:")
    display(df)

    # Display all account IDs
    print("\nAll Account IDs:")
    for account in accounts_result:
        print(f"Account ID: {account.get('id')}, Name: {account.get('name')}")
else:
    print("No accounts found or there was an error fetching the accounts.")

All Accounts:


,lock_reason,unlock_if_subscription_renewed,read_only_seats,explorer_seats,docs_job_id,freshness_job_id,id,name,state,plan,...,identifier,trial_end_date,static_subdomain,run_locked_until,migration_scheduled_at,static_url,semantic_layer_url,semantic_layer_graphql_api_url,semantic_layer_jdbc_url,discovery_api_url
0,None,False,0,0,None,None,257364,Colin T Sandbox Test,1,enterprise,...,act_2flkHTcitUzuEFVbAqr6V2zO0s4,None,None,None,None,https://cloud.getdbt.com/,semantic-layer.cloud.getdbt.com,https://semantic-layer.cloud.getdbt.com/api/gr...,jdbc:arrow-flight-sql://semantic-layer.cloud.g...,https://metadata.cloud.getdbt.com



All Account IDs:
Account ID: 257364, Name: Colin T Sandbox Test


In [25]:
def get_jobs(account_id):
    r = requests.get(f'{cloud_api_base_url}/accounts/{account_id}/jobs', headers=headers)
    jobs_result = r.json()
    return jobs_result.get('data')

jobs = get_jobs(account_id)
display(pd.DataFrame(jobs))

most_recent_job = jobs[0]
job_id = most_recent_job.get("id")
job_name = most_recent_job.get("name")

,execution,generate_docs,run_generate_sources,run_compare_changes,id,account_id,project_id,environment_id,name,description,...,settings,schedule,is_deferrable,job_type,triggers_on_draft_pr,job_completion_trigger_condition,generate_sources,cron_humanized,next_run,next_run_humanized
0,{'timeout_seconds': 0},False,False,False,637765,257364,368293,314743,test job,,...,"{'threads': 4, 'target_name': 'default'}","{'cron': '5 */12 * * 0,1,2,3,4,5,6', 'date': {...",False,other,False,None,False,"At 5 minutes past the hour, every 12 hours, on...",None,None
1,{'timeout_seconds': 0},True,True,False,673346,257364,374609,321463,Production Build On-Demand,,...,"{'threads': 4, 'target_name': 'default'}","{'cron': '5 */12 * * 0,1,2,3,4,5,6', 'date': {...",False,other,False,None,True,"At 5 minutes past the hour, every 12 hours, on...",None,None
2,{'timeout_seconds': 0},False,False,False,673349,257364,374609,321463,Slim CI,,...,"{'threads': 4, 'target_name': 'default'}","{'cron': '5 */12 * * 0,1,2,3,4,5,6', 'date': {...",False,ci,False,None,False,"At 5 minutes past the hour, every 12 hours, on...",None,None
3,{'timeout_seconds': 0},True,True,False,678483,257364,375455,322449,Build on-demand,,...,"{'threads': 4, 'target_name': 'default'}","{'cron': '5 */12 * * 0,1,2,3,4,5,6', 'date': {...",False,other,False,None,True,"At 5 minutes past the hour, every 12 hours, on...",None,None
4,{'timeout_seconds': 0},True,False,False,680060,257364,376029,322946,Production Build,,...,"{'threads': 4, 'target_name': 'default'}","{'cron': '5 */12 * * 0,1,2,3,4,5,6', 'date': {...",False,other,False,None,False,"At 5 minutes past the hour, every 12 hours, on...",None,None
5,{'timeout_seconds': 0},False,False,False,680062,257364,376029,322946,Slim CI,,...,"{'threads': 4, 'target_name': 'default'}","{'cron': '5 */12 * * 0,1,2,3,4,5,6', 'date': {...",False,ci,False,None,False,"At 5 minutes past the hour, every 12 hours, on...",None,None
6,{'timeout_seconds': 0},True,True,False,680069,257364,376089,322949,Production Build (analyst),,...,"{'threads': 4, 'target_name': 'default'}","{'cron': '5 */12 * * 0,1,2,3,4,5,6', 'date': {...",False,other,False,None,True,"At 5 minutes past the hour, every 12 hours, on...",None,None


In [29]:
def trigger_most_recent_job(account_id):
    jobs = get_jobs(account_id)
    if not jobs:
        print("No jobs found.")
        return None

    most_recent_job = jobs[0]
    job_id = most_recent_job.get("id")
    job_name = most_recent_job.get("name")

    url = f"{cloud_api_base_url}/accounts/{account_id}/jobs/{job_id}/run/"
    payload = {
        "cause": "API trigger"
    }
    response = requests.post(url, json=payload, headers=headers)
    result = response.json()

    print(f"Triggered job: {job_name} (ID: {job_id})")
    return result

# Example usage:
result = trigger_most_recent_job(account_id)
if result:
    print("Job trigger result:")
    display(pd.DataFrame([result]))

Triggered job: test job (ID: 637765)
Job trigger result:


,status,data
0,"{'code': 200, 'is_success': True, 'user_messag...","{'id': 309581633, 'trigger_id': 310401203, 'ac..."
